## Imports

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
from pylab import plt, mpl
import mplfinance as mpf
plt.style.use('seaborn-v0_8-notebook')
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(precision=4, suppress=True)
os.environ['PYTHONHASHSEED'] = '0'
%config InlineBackend.figure_format = 'svg'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4' # can also be set in bashrc

from predictor.env import Finance
from predictor.agent import DQLAgent
from predictor.utils import visualize, save_params

In [ ]:
import warnings as w
w.simplefilter('ignore')

In [ ]:
#ticker = ['sinus_noise_std_1'] # must be given as a list, use sinus data to check model 
ticker = ['BTCUSDT']
interval = '5m' # roughly 300 min per period
train_days = 2
test_days = 1
#features = [('log-ret', 'z'), ('ratio_sma_20_40', 'z'), ('hour_otd', 'minmax'), ('volume', 'minmax')]
#features = [('log-ret', 'z'), ('ratio_sma_12_24', 'z'), ('hour_otd', 'minmax')]
#features = [('log-ret', 'z'), ('ratio_sma_20_40', 'z'), ('volume', 'minmax')]
#features = [('log-ret', 'z'), ('ratio_sma_12_24', 'z')]
#features = [('log-ret', 'z'), ('close_5_sma', 'z')]
#features = [('log-ret', 'z'), ('close_12_sma', 'z'), ('close_24_sma', 'z')]
#features = [('log-ret', 'z') , ('hour_otd', 'minmax')]
features = [('log-ret', 'z')]


#features = [('ratio_sma_12_24', 'z')]
#features = [('candle_body', 'z'), ('upper_wick', 'z'), ('lower_wick', 'z')]

lags = 10
END_TEST = date.today() - timedelta(days=1) # data from today is not available
START_TEST = END_TEST - timedelta(days=(test_days-1))
END_TRAIN = START_TEST - timedelta(days=1)
START_TRAIN = END_TRAIN - timedelta(days=(train_days-1)) # -1 because same date is one day already
print(f"Training period: {START_TRAIN} to {END_TRAIN}")
print(f"Testing period: {START_TEST} to {END_TEST}")

In [ ]:
train_env = Finance(ticker, interval, features, lags, min_performance = 0.75, min_sharpe=0.4,
                 start=START_TRAIN, end=END_TRAIN)
train_env.seed(42)

In [ ]:
train_env.raw.head()

In [ ]:
train_env.data['close'].plot(figsize=(13,5))

In [ ]:
test_env = Finance(ticker, interval, features,
                 lags, min_performance=train_env.min_performance,
                 start=START_TEST, end=END_TEST)

In [ ]:
hu = max(lags, 24) # hidden_units
lr = 0.0001 # learing_rate
batch = True
agent = DQLAgent(hu, lr, batch, train_env, test_env)
print('Done')

In [ ]:
agent.value_model.summary()

In [ ]:
episodes = 50
%time agent.learn(episodes)

In [ ]:
# Create a figure and the first y-axis
fig, ax1 = plt.subplots(figsize=(12, 5))

# Data for the first y-axis
x = range(1, len(agent.losses) + 1)
#y = np.polyval(np.polyfit(x, agent.losses, deg=3), x)
ax1.plot(agent.losses, 'bo--', label='loss')
ax1.set_xlabel('episodes')
ax1.set_ylabel('loss', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a second y-axis
ax2 = ax1.twinx()

# Data for the second y-axis
ax2.plot(agent.performances, 'ro--', label='performance')
ax2.set_ylabel('performance', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Loss and Performance on Training Data (with Exploration)')
plt.show()

In [ ]:
actions = []
length = len(test_env.data) - test_env.lags

for i in range(length):
    state = test_env.reset(shift=i)
    state = np.reshape(state, [1, test_env.lags,
                                   test_env.n_features])
    action = agent.value_model.predict(state, verbose=0)[0]
    if i%10==0:
        print(f'progress: {i:>4d}/{length:>4d}', end='\r')
    actions.append(np.argmax(action))

data = pd.DataFrame({
    'date' : test_env.data['date'].tail(length).reset_index(drop=True),
    'close' : test_env.data['close'].tail(length).reset_index(drop=True),
    'log-ret-ns': test_env.data['log-ret-no-scale'].tail(length).reset_index(drop=True),
    'action': actions
})
data['action'].replace(0, -1, inplace=True)
data['performance'] = (data['log-ret-ns'] * data['action']).cumsum().apply(lambda x: np.exp(x))
# paramters to save 
buy_and_hold = (data['close'].iloc[-1] / data['close'].iloc[0] - 1) * 100
perf_on_test = ((data['performance'].iloc[-1]-1) * 100) 

text = 'Buy-and-hold strategy: {:4.1f}% \n'
text += 'DRL strategy out of sample: {:4.1f}%'
print(text.format(buy_and_hold, perf_on_test))

In [ ]:
# Plot performance vs buy-and-hold strategy 
plt.figure(figsize=(12, 6))
plt.plot(data['date'], data['performance'], label='Strategy')
plt.plot(data['date'], data['close'].pct_change().apply(lambda x : x+1).cumprod(), label='Buy and Hold')
plt.title('Strategy vs Buy and Hold')
plt.xlabel('Time')
plt.ylabel('Performance')
plt.legend()
plt.show()

#### Visualize entry and exit points

In [ ]:
#display figure
chart = visualize(data)
chart

In [ ]:
# save figure
figname = 'filename-goes-here.html'
chart.save(figname)

#### Save parameters to file

In [ ]:
filename = 'filename-goes-here.csv'
comment = 'add you comment here'
# Append the parameters to the file
save_params(filename, ticker, test_env.features, lags, episodes, lr, 
            agent.tau, agent.gamma, agent.epsilon_decay, buy_and_hold, perf_on_test, comment)